In [242]:
import pickle
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import gensim
from gensim import corpora
import kss
import time
from konlpy.tag import Mecab
import random

import platform
from collections import Counter

import numpy as np

# tokenizer import
from konlpy.tag import Okt, Komoran, Hannanum, Kkma

#운영체제에 따라 mecab설치 방법이 다름.
if platform.system() == "Windows":
    try:
        from eunjeon import Mecab
    except:
        print("please install eunjeon module")
else:  # Ubuntu일 경우
    from konlpy.tag import Mecab

from typing import List, Callable, Union, Any, TypeVar, Tuple, Dict

from functools import partial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
import gensim
from gensim import corpora
import kss

from gensim.models import LdaModel


In [243]:
with open('economy_id.pickle', 'rb') as f:
    id_news = pickle.load(f)
with open('politic_id.pickle', 'rb') as f:
    id_news.extend(pickle.load(f))
with open('society_id.pickle', 'rb') as f:
    id_news.extend(pickle.load(f))
    

with open('economy_contents.pickle', 'rb') as f:
    news = pickle.load(f)
with open('politic_contents.pickle', 'rb') as f:
    news.extend(pickle.load(f))
with open('society_contents.pickle', 'rb') as f:
    news.extend(pickle.load(f))


In [244]:
len(id_news)
print(len(news))

3908


In [245]:
start = time.time()

print(len(news))
lines = ""

for i in news:
    lines = lines+i


3908


In [246]:
#불용어 사전에 "명" 추가
stop = "순간,탓,운,추천,확정,지지,권고,무근,한때,중요,선점,저게,사이,조사,반발,마무리,시작,결정,계획,알,불편,인,적극,영상제목,스크랩,댓글,전달,적용,구성,최대,포함,상식,목적,종합,수집,추진,노력,지원,작성,오후,공개,전날,인정,해당,부담,의심,선택,단계,사회,유지,발견,발굴,반면,자리,거래,사람,단독,조치,검토,익,의미,초래,강조,최근,추가,논란,확인,자체,대신,실,셈,중심,규모,충격,자체,기사,기자,참석,수준,지속,가운데,지난해,사례,눈치,파악,올해,채,낫,사건,달,개,이해,활용,진행,제외,뉴스,마지막,그게,이게,예정,억,완료,상대,흐름,경우,날,쪽,그,살,소지,주장,후원,손,고,안,때문,걱정,한편,계속,지,사실,백,일보,처음,설명,제공,위,이날,가능,급,앞,식,중앙일보,동아일보,해럴드뉴스,머니투데이,아시아경제,경향일보,국민일보,한겨레,내일뉴스,연합뉴스,세,초,직,뒤,측,데,도,이후,부,대,시,바,외,속,차,점,단,후,듯,곳,필요,기여,이용,만,불,원,중,씨,헤럴드,전재,뿐,면,반,관련,기대,제기,우려,번,지적,배포,금지,말,건,분,회,간,내,수,거,게,명,직전,아,휴,아이구,아이쿠,아이고,어,나,우리,저희,따라,의해,을,를,에,의,가,으로,로,에게,뿐이다,의거하여,근거하여,입각하여,기준으로,예하면,예를 들면,예를 들자면,저,소인,소생,저희,지말고,하지마,하지마라,다른,물론,또한,그리고,비길수 없다,해서는 안된다,뿐만 아니라,만이 아니다,만은 아니다,막론하고,관계없이,그치지 않다,그러나,그런데,하지만,든간에,논하지 않다,따지지 않다,설사,비록,더라도,아니면,만 못하다,하는 편이 낫다,불문하고,향하여,향해서,향하다,쪽으로,틈타,이용하여,타다,오르다,제외하고,이 외에,이 밖에,하여야,비로소,한다면 몰라도,외에도,이곳,여기,부터,기점으로,따라서,할 생각이다,하려고하다,이리하여,그리하여,그렇게 함으로써,하지만,일때,할때,앞에서,중에서,보는데서,으로써,로써,까지,해야한다,일것이다,반드시,할줄알다,할수있다,할수있어,임에 틀림없다,한다면,등,등등,제,겨우,단지,다만,할뿐,딩동,댕그,대해서,대하여,대하면,훨씬,얼마나,얼마만큼,얼마큼,남짓,여,얼마간,약간,다소,좀,조금,다수,몇,얼마,지만,하물며,또한,그러나,그렇지만,하지만,이외에도,대해 말하자면,뿐이다,다음에,반대로,반대로 말하자면,이와 반대로,바꾸어서 말하면,바꾸어서 한다면,만약,그렇지않으면,까악,툭,딱,삐걱거리다,보드득,비걱거리다,꽈당,응당,해야한다,에 가서,각,각각,여러분,각종,각자,제각기,하도록하다,와,과,그러므로,그래서,고로,한 까닭에,하기 때문에,거니와,이지만,대하여,관하여,관한,과연,실로,아니나다를가,생각한대로,진짜로,한적이있다,하곤하였다,하,하하,허허,아하,거바,와,오,왜,어째서,무엇때문에,어찌,하겠는가,무슨,어디,어느곳,더군다나,하물며,더욱이는,어느때,언제,야,이봐,어이,여보시오,흐흐,흥,휴,헉헉,헐떡헐떡,영차,여차,어기여차,끙끙,아야,앗,아야,콸콸,졸졸,좍좍,뚝뚝,주룩주룩,솨,우르르,그래도,또,그리고,바꾸어말하면,바꾸어말하자면,혹은,혹시,답다,및,그에 따르는,때가 되어,즉,지든지,설령,가령,하더라도,할지라도,일지라도,지든지,몇,거의,하마터면,인젠,이젠,된바에야,된이상,만큼,어찌됏든,그위에,게다가,점에서 보아,비추어 보아,고려하면,하게될것이다,일것이다,비교적,좀,보다더,비하면,시키다,하게하다,할만하다,의해서,연이서,이어서,잇따라,뒤따라,뒤이어,결국,의지하여,기대여,통하여,자마자,더욱더,불구하고,얼마든지,마음대로,주저하지 않고,곧,즉시,바로,당장,하자마자,밖에 안된다,하면된다,그래,그렇지,요컨대,다시 말하자면,바꿔 말하면,즉,구체적으로,말하자면,시작하여,시초에,이상,허,헉,허걱,바와같이,해도좋다,해도된다,게다가,더구나,하물며,와르르,팍,퍽,펄렁,동안,이래,하고있었다,이었다,에서,로부터,까지,예하면,했어요,해요,함께,같이,더불어,마저,마저도,양자,모두,습니다,가까스로,하려고하다,즈음하여,다른,다른 방면으로,해봐요,습니까,했어요,말할것도 없고,무릎쓰고,개의치않고,하는것만 못하다,하는것이 낫다,매,매번,들,모,어느것,어느,로써,갖고말하자면,어디,어느쪽,어느것,어느해,어느 년도,라 해도,언젠가,어떤것,어느것,저기,저쪽,저것,그때,그럼,그러면,요만한걸,그래,그때,저것만큼,그저,이르기까지,할 줄 안다,할 힘이 있다,너,너희,당신,어찌,설마,차라리,할지언정,할지라도,할망정,할지언정,구토하다,게우다,토하다,메쓰겁다,옆사람,퉤,쳇,의거하여,근거하여,의해,따라,힘입어,그,다음,버금,두번째로,기타,첫번째로,나머지는,그중에서,견지에서,형식으로 쓰여,입장에서,위해서단지,의해되다,하도록시키다,뿐만아니라,반대로,전후,전자,앞의것,잠시,잠깐,하면서,그렇지만,다음에,그러한즉,그런즉,남들,아무거나,어찌하든지,같다,비슷하다,예컨대,이럴정도로,어떻게,만약,만일,위에서 서술한바와같이,인 듯하다,하지 않는다면,만약에,무엇,무슨,어느,어떤,아래윗,조차,한데,그럼에도 불구하고,여전히,심지어,까지도,조차도,하지 않도록,않기 위하여,때,시각,무렵,시간,동안,어때,어떠한,하여금,네,예,우선,누구,누가 알겠는가,아무도,줄은모른다,줄은 몰랏다,하는 김에,겸사겸사,하는바,그런 까닭에,한 이유는,그러니,그러니까,때문에,그,너희,그들,너희들,타인,것,것들,너,위하여,공동으로,동시에,하기 위하여,어찌하여,무엇때문에,붕붕,윙윙,나,우리,엉엉,휘익,윙윙,오호,아하,어쨋든,만 못하다,하기보다는,차라리,하는 편이 낫다,흐흐,놀라다,상대적으로 말하자면,마치,아니라면,쉿,그렇지 않으면,그렇지 않다면,안 그러면,아니었다면,하든지,아니면,이라면,좋아,알았어,하는것도,그만이다,어쩔수 없다,하나,일,일반적으로,일단,한켠으로는,오자마자,이렇게되면,이와같다면,전부,한마디,한항목,근거로,하기에,아울러,하지 않도록,않기 위해서,이르기까지,이 되다,로 인하여,까닭으로,이유만으로,이로 인하여,그래서,이 때문에,그러므로,그런 까닭에,알 수 있다,결론을 낼 수 있다,으로 인하여,있다,어떤것,관계가 있다,관련이 있다,연관되다,어떤것들,에 대해,이리하여,그리하여,여부,하기보다는,하느니,하면 할수록,운운,이러이러하다,하구나,하도다,다시말하면,다음으로,에 있다,에 달려 있다,우리,우리들,오히려,하기는한데,어떻게,어떻해,어찌됏어,어때,어째서,본대로,자,이,이쪽,여기,이것,이번,이렇게말하자면,이런,이러한,이와 같은,요만큼,요만한 것,얼마 안 되는 것,이만큼,이 정도의,이렇게 많은 것,이와 같다,이때,이렇구나,것과 같이,끼익,삐걱,따위,와 같은 사람들,부류의 사람들,왜냐하면,중의하나,오직,오로지,에 한하다,하기만 하면,도착하다,까지 미치다,도달하다,정도에 이르다,할 지경이다,결과에 이르다,관해서는,여러분,하고 있다,한 후,혼자,자기,자기집,자신,우에 종합한것과같이,총적으로 보면,총적으로 말하면,총적으로,대로 하다,으로서,참,그만이다,할 따름이다,쿵,탕탕,쾅쾅,둥둥,봐,봐라,아이야,아니,와아,응,아이,참나,년,월,일,령,영,일,이,삼,사,오,육,륙,칠,팔,구,이천육,이천칠,이천팔,이천구,하나,둘,셋,넷,다섯,여섯,일곱,여덟,아홉,령,영 "

In [247]:
news_doc = lines

stime = time.time()
text_list = kss.split_sentences(news_doc)
print(time.time()-stime)
k = []


49.60196614265442


In [248]:
stop_words = stop.split(',')
mecab = Mecab()
stime = time.time()
for i in news :
    sentences = mecab.nouns(i)
    result2 = []
    for word in sentences :
        if word not in stop_words:
            result2.append(word)
    k.append(result2)
print(time.time()-stime)
print(len(k))

22.495360136032104
3908


In [249]:
bigram = gensim.models.Phrases(k, min_count = 5, threshold = 1)

bigram_model = gensim.models.phrases.Phraser(bigram)

bigram_document = [bigram_model[nouns] for nouns in k]

for i in range(len(bigram_document)):
    if word in stop_words:
        bigram_document.drop(i)



In [250]:
id2word = corpora.Dictionary(bigram_document)
corpus = [id2word.doc2bow(doc) for doc in bigram_document]

In [251]:
from gensim.models import CoherenceModel

coherence_score=[]
t_min = 2
t_max = 20
for i in range(t_min,t_max):
    model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=i)
    coherence_model = CoherenceModel(model, texts=bigram_document, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model.get_coherence()
    print('n=',i,'\nCoherence Score: ', coherence_lda)
    coherence_score.append(coherence_lda)


n= 2 
Coherence Score:  0.31176035988291556
n= 3 
Coherence Score:  0.2098885618128712
n= 4 
Coherence Score:  0.3015231765467976
n= 5 
Coherence Score:  0.3672053684152908
n= 6 
Coherence Score:  0.36375708566967724
n= 7 
Coherence Score:  0.27148671908814326
n= 8 
Coherence Score:  0.4458559850063036
n= 9 
Coherence Score:  0.417811251990327
n= 10 
Coherence Score:  0.41343042646891287
n= 11 
Coherence Score:  0.39857826030680527
n= 12 
Coherence Score:  0.4048318541644762
n= 13 
Coherence Score:  0.3851963588931047
n= 14 
Coherence Score:  0.4973065954312433
n= 15 
Coherence Score:  0.4095109434677511
n= 16 
Coherence Score:  0.4006352465069327
n= 17 
Coherence Score:  0.3631706550171662
n= 18 
Coherence Score:  0.3744749874678782
n= 19 
Coherence Score:  0.3566936232237721


In [252]:

co_sc = np.array(coherence_score)
NUM_TOPICS = np.argmax(co_sc)+t_min

In [253]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=id2word, passes=15)
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.006*"기업" + 0.006*"시장" + 0.005*"정부" + 0.004*"투자" + 0.004*"놀이터"')
(1, '0.011*"평양_영상" + 0.010*"이용구_폭행" + 0.010*"대국민_사기극" + 0.010*"평양_지도" + 0.010*"초간_고민"')
(2, '0.009*"군" + 0.008*"군사_경찰" + 0.007*"중사" + 0.007*"장_중사" + 0.007*"피해자"')
(3, '0.007*"경찰" + 0.005*"학생" + 0.004*"여성" + 0.003*"문제" + 0.003*"사고"')
(4, '0.005*"사업" + 0.005*"한국" + 0.005*"지역" + 0.005*"환경" + 0.004*"사용"')
(5, '0.016*"아시아_경제" + 0.014*"블로그_주소" + 0.014*"복사_이미지" + 0.014*"블로그_네이버" + 0.014*"세계_창"')
(6, '0.012*"수사" + 0.008*"부분" + 0.007*"문제" + 0.007*"차관" + 0.006*"앵커"')
(7, '0.016*"국민_힘" + 0.013*"후보" + 0.010*"정치" + 0.009*"의원" + 0.007*"지사"')
(8, '0.007*"문_대통령" + 0.007*"북한" + 0.007*"부회장" + 0.006*"기업" + 0.005*"부회장_사면"')
(9, '0.010*"정부" + 0.005*"코로나" + 0.004*"재난_지원금" + 0.004*"상황" + 0.004*"지역"')
(10, '0.008*"사업" + 0.007*"기업" + 0.005*"분야" + 0.005*"대학" + 0.005*"운영"')
(11, '0.008*"시장" + 0.006*"상승" + 0.005*"부동산" + 0.004*"서울_아파트" + 0.004*"주"')
(12, '0.009*"달러" + 0.006*"테슬라" + 0.004*"회사" + 0.004*"차량" + 0.004*"택배"')
(13, '0.024*"접종" + 0.019

In [254]:

def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob


In [255]:
lda_model = LdaModel(corpus, id2word=id2word, num_topics=NUM_TOPICS)

In [256]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(14,)
1.0000001
(14, 38109)
1.0


In [257]:
import pyLDAvis.gensim_models as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, id2word)

In [258]:
import pyLDAvis

In [259]:
pyLDAvis.display(prepared_data)

In [260]:
for i, topic_list in enumerate(ldamodel[corpus]):
    print(id_news[i],'번째 문서의 topic 비율은',topic_list)

4923 번째 문서의 topic 비율은 [(4, 0.100995526), (5, 0.5669419), (10, 0.3241184)]
4926 번째 문서의 topic 비율은 [(0, 0.27627102), (5, 0.19361806), (8, 0.14837846), (9, 0.37782183)]
4929 번째 문서의 topic 비율은 [(0, 0.60363495), (5, 0.14255553), (9, 0.22733165), (12, 0.023651624)]
4932 번째 문서의 topic 비율은 [(0, 0.048072957), (5, 0.29646602), (10, 0.6504493)]
4935 번째 문서의 topic 비율은 [(5, 0.1895269), (10, 0.03975739), (11, 0.7660341)]
4938 번째 문서의 topic 비율은 [(5, 0.1363079), (10, 0.23115477), (11, 0.6287926)]
4941 번째 문서의 topic 비율은 [(5, 0.15470938), (9, 0.78053546), (10, 0.06114008)]
4944 번째 문서의 topic 비율은 [(5, 0.48093185), (11, 0.408389), (12, 0.10517778)]
4947 번째 문서의 topic 비율은 [(5, 0.100277066), (10, 0.89602506)]
4950 번째 문서의 topic 비율은 [(4, 0.70454234), (5, 0.28679132)]
4953 번째 문서의 topic 비율은 [(13, 0.9927972)]
4956 번째 문서의 topic 비율은 [(5, 0.18694796), (6, 0.07299752), (8, 0.3490394), (11, 0.37233254), (13, 0.014657247)]
4959 번째 문서의 topic 비율은 [(5, 0.14722371), (10, 0.847449)]
4962 번째 문서의 topic 비율은 [(0, 0.79265684), (13, 0.2

5811 번째 문서의 topic 비율은 [(3, 0.7701563), (4, 0.063661985), (10, 0.15921097)]
5814 번째 문서의 topic 비율은 [(4, 0.99336165)]
5817 번째 문서의 topic 비율은 [(12, 0.9937217)]
5820 번째 문서의 topic 비율은 [(4, 0.17052913), (12, 0.8255138)]
5823 번째 문서의 topic 비율은 [(4, 0.61344755), (8, 0.33760288), (9, 0.04254444)]
5826 번째 문서의 topic 비율은 [(11, 0.989193)]
5829 번째 문서의 topic 비율은 [(4, 0.30785424), (9, 0.08654254), (10, 0.5985103)]
5832 번째 문서의 topic 비율은 [(0, 0.25421545), (10, 0.33769214), (12, 0.028870877), (13, 0.37151775)]
5835 번째 문서의 topic 비율은 [(0, 0.4523627), (3, 0.012632172), (6, 0.23806705), (8, 0.16828425), (11, 0.123583175)]
5838 번째 문서의 topic 비율은 [(10, 0.31306428), (13, 0.6813626)]
5841 번째 문서의 topic 비율은 [(0, 0.36790508), (6, 0.1774369), (10, 0.4351492), (11, 0.016522888)]
5844 번째 문서의 topic 비율은 [(7, 0.16805725), (9, 0.1585862), (11, 0.6690934)]
5847 번째 문서의 topic 비율은 [(2, 0.6105106), (4, 0.38307574)]
5850 번째 문서의 topic 비율은 [(4, 0.4169774), (9, 0.32211834), (10, 0.25041434)]
5853 번째 문서의 topic 비율은 [(9, 0.7735482), (10,

6441 번째 문서의 topic 비율은 [(0, 0.53891367), (13, 0.43963644)]
6444 번째 문서의 topic 비율은 [(1, 0.79755026), (4, 0.11370773), (8, 0.022252621), (12, 0.057962913)]
6447 번째 문서의 topic 비율은 [(0, 0.046857506), (5, 0.2345427), (9, 0.53436905), (12, 0.17493355)]
6450 번째 문서의 topic 비율은 [(11, 0.17025973), (12, 0.82472235)]
6453 번째 문서의 topic 비율은 [(0, 0.09169053), (10, 0.41947958), (11, 0.029218297), (12, 0.45650148)]
6456 번째 문서의 topic 비율은 [(0, 0.25261423), (3, 0.03442256), (4, 0.42242306), (9, 0.28576174)]
6459 번째 문서의 topic 비율은 [(0, 0.5855432), (4, 0.39794132)]
6462 번째 문서의 topic 비율은 [(10, 0.14714798), (12, 0.8458203)]
6465 번째 문서의 topic 비율은 [(4, 0.44261783), (11, 0.5466424)]
6468 번째 문서의 topic 비율은 [(3, 0.06628097), (4, 0.5334823), (13, 0.39194223)]
6471 번째 문서의 topic 비율은 [(4, 0.05311175), (10, 0.59573823), (12, 0.3413147)]
6474 번째 문서의 topic 비율은 [(8, 0.99244404)]
6477 번째 문서의 topic 비율은 [(3, 0.03567098), (7, 0.11708346), (8, 0.085917726), (9, 0.15049152), (10, 0.4938862), (13, 0.10636205)]
6480 번째 문서의 topic 비율은 [(

7224 번째 문서의 topic 비율은 [(0, 0.28436628), (5, 0.19701077), (8, 0.14964163), (9, 0.36504927)]
7227 번째 문서의 topic 비율은 [(0, 0.6036366), (5, 0.14255577), (9, 0.2273292), (12, 0.023652133)]
7230 번째 문서의 topic 비율은 [(0, 0.04806751), (5, 0.29646668), (10, 0.65045416)]
7233 번째 문서의 topic 비율은 [(5, 0.18952017), (10, 0.039775945), (11, 0.7660223)]
7236 번째 문서의 topic 비율은 [(5, 0.13630782), (10, 0.2311553), (11, 0.62879217)]
7239 번째 문서의 topic 비율은 [(5, 0.15470989), (9, 0.7805383), (10, 0.061136782)]
7242 번째 문서의 topic 비율은 [(5, 0.48092964), (11, 0.4083745), (12, 0.105194524)]
7245 번째 문서의 topic 비율은 [(5, 0.10027711), (10, 0.896025)]
7248 번째 문서의 topic 비율은 [(4, 0.7045422), (5, 0.28679147)]
7251 번째 문서의 topic 비율은 [(13, 0.9927972)]
7254 번째 문서의 topic 비율은 [(5, 0.18694738), (6, 0.07299972), (8, 0.3490379), (11, 0.37233093), (13, 0.0146587435)]
7257 번째 문서의 topic 비율은 [(5, 0.1472238), (10, 0.84744895)]
7260 번째 문서의 topic 비율은 [(0, 0.79266185), (13, 0.20207734)]
7263 번째 문서의 topic 비율은 [(9, 0.7410324), (11, 0.045140542), (13, 

7794 번째 문서의 topic 비율은 [(0, 0.05683235), (10, 0.2321594), (11, 0.31007203), (13, 0.39640838)]
7797 번째 문서의 topic 비율은 [(0, 0.034386855), (10, 0.8782781), (12, 0.07303395)]
7800 번째 문서의 topic 비율은 [(4, 0.70675755), (13, 0.28641924)]
7803 번째 문서의 topic 비율은 [(4, 0.9917005)]
7806 번째 문서의 topic 비율은 [(0, 0.3707046), (3, 0.04596671), (4, 0.5295426), (9, 0.02107261), (13, 0.028042682)]
7809 번째 문서의 topic 비율은 [(4, 0.9888034)]
7812 번째 문서의 topic 비율은 [(4, 0.99354523)]
7815 번째 문서의 topic 비율은 [(9, 0.57901347), (11, 0.41629627)]
7818 번째 문서의 topic 비율은 [(0, 0.10443234), (3, 0.071820594), (4, 0.4914625), (10, 0.069046125), (13, 0.25719863)]
7821 번째 문서의 topic 비율은 [(0, 0.013749358), (5, 0.9793325)]
7824 번째 문서의 topic 비율은 [(9, 0.9962227)]
7827 번째 문서의 topic 비율은 [(0, 0.14737874), (2, 0.04519788), (3, 0.04283379), (11, 0.15937947), (12, 0.60217345)]
7830 번째 문서의 topic 비율은 [(9, 0.668364), (11, 0.3264052)]
7833 번째 문서의 topic 비율은 [(11, 0.9951862)]
7836 번째 문서의 topic 비율은 [(0, 0.3938345), (1, 0.24798779), (4, 0.11978116), (10,

...

5794 번째 문서의 topic 비율은 [(1, 0.32067224), (3, 0.058567457), (4, 0.12115173), (6, 0.09727988), (10, 0.022575317), (11, 0.051473718), (12, 0.07378026), (13, 0.25157607)]
5797 번째 문서의 topic 비율은 [(1, 0.62581915), (4, 0.3321642), (11, 0.02561816)]
5800 번째 문서의 topic 비율은 [(1, 0.380806), (4, 0.48375666), (6, 0.0873379), (10, 0.03904241)]
5803 번째 문서의 topic 비율은 [(1, 0.09516303), (3, 0.11712537), (4, 0.42029688), (7, 0.04039963), (9, 0.06967913), (10, 0.2554472)]
5806 번째 문서의 topic 비율은 [(0, 0.09292225), (1, 0.35455957), (3, 0.095664196), (4, 0.1966802), (8, 0.20233978), (9, 0.052431684)]
5809 번째 문서의 topic 비율은 [(0, 0.03663213), (1, 0.3851672), (3, 0.15601854), (4, 0.1318398), (9, 0.26353744), (13, 0.021506371)]
5812 번째 문서의 topic 비율은 [(0, 0.0657402), (1, 0.13308643), (4, 0.7327155), (11, 0.01115868), (12, 0.053674668)]
5815 번째 문서의 topic 비율은 [(0, 0.033775765), (1, 0.12699863), (4, 0.12879053), (7, 0.033132587), (8, 0.09705551), (10, 0.5331384), (12, 0.045590974)]
5818 번째 문서의 topic 비율은 [(0, 0.048456952),

6545 번째 문서의 topic 비율은 [(0, 0.052900515), (3, 0.066979945), (4, 0.33086222), (10, 0.5008887), (11, 0.04386567)]
6547 번째 문서의 topic 비율은 [(3, 0.4972893), (4, 0.25408202), (5, 0.01822688), (6, 0.029322512), (7, 0.051559407), (9, 0.07870603), (10, 0.038908865), (13, 0.028850365)]
6549 번째 문서의 topic 비율은 [(0, 0.020794867), (4, 0.21296787), (7, 0.021304714), (9, 0.12027816), (10, 0.621644)]
6551 번째 문서의 topic 비율은 [(1, 0.21010557), (2, 0.022402855), (3, 0.08976802), (5, 0.025219066), (6, 0.6255166), (7, 0.020804271)]
6553 번째 문서의 topic 비율은 [(3, 0.08410826), (7, 0.17275792), (9, 0.1284357), (10, 0.03915522), (13, 0.5720988)]
6555 번째 문서의 topic 비율은 [(2, 0.07898069), (3, 0.69718486), (6, 0.17398171), (11, 0.012469384), (13, 0.024977619)]
6557 번째 문서의 topic 비율은 [(1, 0.0568133), (2, 0.10717018), (3, 0.69695497), (4, 0.12382747)]
6559 번째 문서의 topic 비율은 [(1, 0.35527658), (4, 0.36213702), (9, 0.27355427)]
6561 번째 문서의 topic 비율은 [(0, 0.04560586), (3, 0.89104104), (6, 0.048224144)]
6563 번째 문서의 topic 비율은 [(0, 0.0

6969 번째 문서의 topic 비율은 [(3, 0.82632035), (8, 0.14068593)]
6970 번째 문서의 topic 비율은 [(3, 0.09113485), (5, 0.8050974), (9, 0.019211115), (10, 0.07686264)]
6971 번째 문서의 topic 비율은 [(1, 0.29674563), (3, 0.44308993), (5, 0.07234735), (7, 0.17359161)]
6972 번째 문서의 topic 비율은 [(0, 0.65396166), (5, 0.113479756), (13, 0.1911832)]
6973 번째 문서의 topic 비율은 [(0, 0.62562567), (7, 0.10496395), (13, 0.23010324)]
6974 번째 문서의 topic 비율은 [(1, 0.14283456), (5, 0.06145182), (6, 0.77814996), (7, 0.010181472)]
6975 번째 문서의 topic 비율은 [(0, 0.11220243), (3, 0.675419), (5, 0.032042142), (7, 0.06285155), (8, 0.044021778), (10, 0.06695543)]
6976 번째 문서의 topic 비율은 [(3, 0.12017374), (6, 0.0845741), (9, 0.020222882), (10, 0.032672886), (13, 0.7343119)]
6977 번째 문서의 topic 비율은 [(0, 0.030476358), (3, 0.1560874), (4, 0.47342613), (6, 0.036436718), (9, 0.07944568), (12, 0.22033472)]
6978 번째 문서의 topic 비율은 [(1, 0.096996516), (2, 0.100148365), (3, 0.7303051), (13, 0.04502338)]
6979 번째 문서의 topic 비율은 [(3, 0.23822455), (4, 0.34326184), (9, 0

7348 번째 문서의 topic 비율은 [(4, 0.51513696), (5, 0.11064273), (10, 0.28773057), (11, 0.032568198), (12, 0.051510066)]
7351 번째 문서의 topic 비율은 [(1, 0.1890992), (3, 0.15884607), (5, 0.4556253), (6, 0.07186684), (7, 0.11806146)]
7354 번째 문서의 topic 비율은 [(3, 0.0191542), (5, 0.3648802), (13, 0.60488915)]
7357 번째 문서의 topic 비율은 [(3, 0.046824418), (5, 0.18789437), (9, 0.43808833), (10, 0.2736538), (13, 0.04906797)]
7360 번째 문서의 topic 비율은 [(4, 0.03373959), (6, 0.0123659065), (9, 0.16577008), (13, 0.7761328)]
7363 번째 문서의 topic 비율은 [(4, 0.5146423), (5, 0.2920914), (10, 0.145136), (11, 0.0169294), (12, 0.02506838)]
7366 번째 문서의 topic 비율은 [(0, 0.09847165), (5, 0.4097471), (9, 0.073898025), (10, 0.03716392), (13, 0.37065658)]
7369 번째 문서의 topic 비율은 [(4, 0.017480213), (5, 0.23871274), (10, 0.07667575), (12, 0.6611052)]
7372 번째 문서의 topic 비율은 [(3, 0.17254932), (4, 0.08229883), (5, 0.34010997), (9, 0.031445336), (10, 0.35475054), (12, 0.012877002)]
7375 번째 문서의 topic 비율은 [(2, 0.040681925), (3, 0.052457444), (5, 0.52

8032 번째 문서의 topic 비율은 [(0, 0.15157287), (1, 0.100549914), (3, 0.21143937), (6, 0.12014697), (7, 0.4132686)]
8035 번째 문서의 topic 비율은 [(0, 0.034625012), (4, 0.2630437), (8, 0.21553855), (10, 0.42902577), (12, 0.052446555)]
8038 번째 문서의 topic 비율은 [(4, 0.3552829), (9, 0.018890616), (10, 0.4821398), (11, 0.036992114), (12, 0.10191667)]
8041 번째 문서의 topic 비율은 [(3, 0.6944043), (4, 0.1816969), (10, 0.11367503)]
8044 번째 문서의 topic 비율은 [(0, 0.5489443), (2, 0.08013434), (3, 0.15909582), (6, 0.041900136), (8, 0.019915916), (9, 0.07908588), (13, 0.0647418)]
8047 번째 문서의 topic 비율은 [(0, 0.069484204), (3, 0.037740372), (9, 0.31431106), (13, 0.56905866)]
8050 번째 문서의 topic 비율은 [(1, 0.07778657), (2, 0.040525407), (3, 0.80128527), (6, 0.057905324), (10, 0.01605004)]
8053 번째 문서의 topic 비율은 [(0, 0.72971535), (2, 0.11803375), (6, 0.047407195), (9, 0.028035674), (13, 0.06875817)]
8056 번째 문서의 topic 비율은 [(1, 0.053436596), (2, 0.07288923), (3, 0.7429264), (4, 0.04223936), (13, 0.08343835)]
8059 번째 문서의 topic 비율은 [(1, 0.

In [261]:
pp = [[]]*(NUM_TOPICS+1)#NO Topic0
for i in range(NUM_TOPICS):
    pp[i] = prepared_data.sorted_terms(topic = i,_lambda=0.6).iloc[:20]

In [262]:
#index만 추출하기(index가 corpus의 index랑 동일함 ㅇㅇ)
idx_topic = [[]]*(NUM_TOPICS+1)
for i in range(1,len(pp)-1):
    idx_topic[i] = pp[i].iloc[:20].index

In [263]:
for i in idx_topic[0]:
    print(id2word[i])

In [264]:

#topic별 clustering을 위한 topic_docs 초기화(list[[]]*n 이런 식으로 하면 안됨)
topic_docs = []
topic_docs_save = []
for i in range(NUM_TOPICS+1):
    topic_docs.append([])
    topic_docs_save.append([])

In [265]:
#append doc_index in the topic_docs cluster
for i, topic_list in enumerate(ldamodel[corpus]):
    topic_list.sort(reverse=True,key = lambda element:element[1])
    n = topic_list[0][0] + 1
    topic_docs[n].append([i,topic_list[0][1]])


In [266]:
#append doc_index in the topic_docs cluster
for i, topic_list in enumerate(ldamodel[corpus]):
    topic_list.sort(reverse=True,key = lambda element:element[1])
    n = topic_list[0][0] + 1
    topic_docs_save[n].append([id_news[i],topic_list[0][1]])


In [267]:
for i in topic_docs:
    i.sort(reverse=True,key = lambda element:element[1])


In [268]:
topic_cluster = []
for i in range(NUM_TOPICS+1):
    topic_cluster.append("")
for i in range(1,NUM_TOPICS+1):
    for j in topic_docs[i]:
        topic_cluster[i] = topic_cluster[i]+news[j[0]]
    

In [269]:
for i, topic_list in enumerate(ldamodel[corpus]):
    topic_list.sort(reverse=True,key= lambda element:element[1])
    #print(i,'번째 문서의 topic 비율은',topic_list)

In [270]:
topic_cluster_sentences = []
tcs = []
for i in range(NUM_TOPICS+1):
    tcs.append([])
    topic_cluster_sentences.append([])

In [271]:
for i in range(1,NUM_TOPICS+1):
    mecab = Mecab()
    stop_words = stop.split(',')
    topic_cluster_sentences[i] = kss.split_sentences(topic_cluster[i])
    for j in topic_cluster_sentences[i]:
        sen_word = []
        sentences = mecab.nouns(j)
        for word in sentences :
            if word not in stop_words:
                sen_word.append(word)
        tcs[i].append(sen_word)

In [272]:
bigram_docs = []
for i in range(NUM_TOPICS+1):
    bigram_docs.append([])
#construct bigram_docs --> use the model made before
for i in range(1,len(tcs)):
    bigram_docs[i] = [bigram_model[nouns] for nouns in tcs[i]]

In [273]:
corpus_docs = []
for i in range(NUM_TOPICS+1):
    corpus_docs.append([])
for i in range(1,NUM_TOPICS+1):    
    corpus_docs[i] = [id2word.doc2bow(doc) for doc in bigram_docs[i]]

In [274]:
corp_doc_ref = []
for i in range(NUM_TOPICS+1):
    corp_doc_ref.append([])


In [275]:
stime = time.time()

for i in range(1,NUM_TOPICS+1):
    if len(corpus_docs[i]) is not 0:
        for j in range(len(corpus_docs[i])):
            corp_doc_ref[i].append([])
            for k in range(len(corpus_docs[i][j])):
                a = corpus_docs[i][j][k][0]
                corp_doc_ref[i][j].append(a)

print(time.time()-stime)

0.1797502040863037


In [276]:
#키워드 포함된 문장만 추출
stime = time.time()
corp_doc_topic = []
for i in range(NUM_TOPICS+1):
    corp_doc_topic.append([])
for i in range(1,len(corp_doc_ref)):
    for j in range(len(corp_doc_ref[i])):
        if len(set(idx_topic[i]).intersection(corp_doc_ref[i][j])) is not 0:
            corp_doc_topic[i].append(bigram_docs[i][j])

print(time.time()-stime)

0.06453299522399902


In [277]:

print(corp_doc_topic[2][4])
for i in idx_topic[2]:
    print(id2word[i])


['서울_뉴시스', '창']
경찰
공직자_양심
평양_지도
평양_영상
인격_국민
김형석_칼럼
대국민_사기극
개편_반대
창
초간_고민
제목_판매
박범계_김오수
영상_경관
이용구_폭행
송영길_조국
징역_선고
경고_이용구
채용_前
혐의
북한


In [278]:
def vectorize_sents(corp_doc_topic = corp_doc_topic, min_count=2, tokenizer="mecab", noun=True):
   

    vectorizer = CountVectorizer(tokenizer=lambda x: x,lowercase=False)

    vec = vectorizer.fit_transform(corp_doc_topic)
    vocab_idx = vectorizer.vocabulary_
    idx_vocab = {idx: vocab for vocab, idx in vocab_idx.items()}
    
    return vec, vocab_idx, idx_vocab

In [279]:
def word_similarity_matrix(x, min_sim=0.3):
    sim_mat = 1 - pairwise_distances(x.T, metric="cosine")
    sim_mat[np.where(sim_mat <= min_sim)] = 0

    return sim_mat

In [280]:
def word_graph(
    corp_doc_topic = corp_doc_topic,
    min_count=2,
    min_sim=0.3,
    tokenizer="mecab",
    noun=True,
):

    mat, vocab_idx, idx_vocab = vectorize_sents(
        corp_doc_topic,min_count=min_count, tokenizer=tokenizer, noun=noun
    )

    mat = word_similarity_matrix(mat, min_sim=min_sim)

    return mat, vocab_idx, idx_vocab

In [281]:
def pagerank(x: np.ndarray, df=0.85, max_iter=50, method="iterative"):

    assert 0 < df < 1

    A = normalize(mat, axis=0, norm="l1")
    N = np.ones(A.shape[0]) / A.shape[0]

    if method == "iterative":
        R = np.ones(A.shape[0])
        for _ in range(max_iter):
            R = df * np.matmul(A, R) + (1 - df) * N
    elif method == "algebraic":
        R = np.linalg.inv((I - df * A))
        R = np.matmul(R, (1 - df) * N)

    return R

In [282]:
keywords = []

In [283]:
for i in range(NUM_TOPICS+1):
    if len(corp_doc_topic[i])!= 0:
        mat,vocab_idx, idx_vocab = word_graph(corp_doc_topic[i])
        R = pagerank(mat,method ="iterative")
        topk = 10
        idxs = R.argsort()[-topk:]
        keywords.append([(idx, R[idx], idx_vocab[idx]) for idx in reversed(idxs)])
        #keyword = [(R[idx]) for idx in reversed(idxs)]


In [284]:
keywords

[[(2179, 0.0009950638950734828, '스마트'),
  (1433, 0.0009856580530812008, '방침'),
  (3495, 0.0009824793899138769, '중소기업'),
  (1681, 0.0009714071347685349, '분석'),
  (1193, 0.0009707746698493099, '매출'),
  (689, 0.0009565901042745041, '기존'),
  (1035, 0.000952642306122776, '돈'),
  (2652, 0.000952642306122776, '외환_위기'),
  (1400, 0.000952642306122776, '방송'),
  (651, 0.0009502152789713444, '기술')],
 [(1729, 0.0014446534226085852, '대전_지검'),
  (4960, 0.0014226687736234175, '인사_검찰'),
  (7318, 0.001422562220863194, '형사_부장'),
  (3614, 0.001418518677380831, '수사'),
  (6923, 0.001414001670931007, '평가'),
  (4853, 0.0014059857181062691, '이성윤_서울'),
  (430, 0.001405391359827535, '검찰_총장'),
  (7479, 0.0013890818776511732, '후보_거론'),
  (416, 0.0013863698155977535, '검찰_내부'),
  (4957, 0.001385871346626384, '인사')],
 [(630, 0.0019788122251438345, '부대'),
  (1787, 0.00187818062114597, '확대'),
  (377, 0.0018178171146888804, '대대'),
  (386, 0.0018149680617901198, '대상'),
  (1099, 0.0018112593994579309, '위주'),
  (602, 0.001

In [285]:
for i in keywords:
    for j in range(5):
        print(i[j][2], end = ",")
    print("")

스마트,방침,중소기업,분석,매출,
대전_지검,인사_검찰,형사_부장,수사,평가,
부대,확대,대대,대상,위주,
힘,서울_중구,프로세스,산업_재해,팝,
배,강화,국가,해직_교사,채용_前,
시행,걸로_조희연,해직_교사,김경율_검찰,경고_이용구,
대통령,박범계_김오수,개편_반대,군검찰,문제,
감사,지사,관심,전,그것,
사진,지난달,입장,방문,정부,
개편_반대,박범계_김오수,어려움,시기,모습,
돈,구청장,생산,유럽,이거,
대상,하락,국민_무단,전,최고_기록,
문화,선,대부분,소식,기회,


In [286]:
end = time.time()

In [287]:
print(end-start)

444.70932507514954


In [288]:
def save_topics():
    ext_topic_cluster = dict()
    tc = []
    for i in range(1,NUM_TOPICS):
        top_save = dict()
        for j in range(len(topic_docs_save[i])):
            top_save[topic_docs_save[i][j][0]] = topic_docs_save[i][j][1]
        ext_topic_cluster[i] = [keywords[i-1],top_save]
    return ext_topic_cluster, NUM_TOPICS

In [289]:
keywords_docs_save,_ = save_topics()

In [290]:
keywords_docs_save

{1: [[(2179, 0.0009950638950734828, '스마트'),
   (1433, 0.0009856580530812008, '방침'),
   (3495, 0.0009824793899138769, '중소기업'),
   (1681, 0.0009714071347685349, '분석'),
   (1193, 0.0009707746698493099, '매출'),
   (689, 0.0009565901042745041, '기존'),
   (1035, 0.000952642306122776, '돈'),
   (2652, 0.000952642306122776, '외환_위기'),
   (1400, 0.000952642306122776, '방송'),
   (651, 0.0009502152789713444, '기술')],
  {4929: 0.6036408,
   4962: 0.7926624,
   4989: 0.6362313,
   5070: 0.550294,
   5094: 0.8065739,
   5097: 0.5580236,
   5130: 0.4187897,
   5346: 0.7404138,
   5355: 0.78151244,
   5361: 0.73997325,
   5364: 0.5791061,
   5397: 0.64302546,
   5433: 0.31195384,
   5439: 0.5428861,
   5526: 0.31858265,
   5541: 0.61591077,
   5574: 0.9040052,
   5577: 0.8696511,
   5646: 0.99139774,
   5673: 0.6194273,
   5679: 0.47164735,
   5706: 0.52622586,
   5709: 0.9920595,
   5730: 0.49383476,
   5784: 0.9941931,
   5799: 0.60738885,
   5808: 0.5959573,
   5835: 0.45238295,
   5865: 0.30158246,
   5

In [291]:
with open("key_doc.pickle","wb") as f:
    pickle.dump(keywords_docs_save,f)